# Imports

In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# import torchvision.io as io
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision.models import ResNet18_Weights, resnet18
from tqdm.auto import tqdm, trange

import wandb

wandb.init(project="hairstyle")
%load_ext autoreload
%autoreload 2

wandb: Currently logged in as: astadnik. Use `wandb login --relogin` to force relogin


# Load data

In [2]:
data_path = "/content/drive/MyDrive/UCU/CV/HW5_Project/data/"
data_path = "data"
ann_path = os.path.join(data_path, "validation_annotation.csv")
annotation_df = pd.read_csv(ann_path)

In [3]:
train_df, val_df = train_test_split(annotation_df, test_size=0.2)
train_df, val_df = train_df.reset_index(drop=True), val_df.reset_index(drop=True)

In [4]:
any([v is None for v in annotation_df.basestyle.values])

False

In [5]:
class HairStyleDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform=None):
        # translator = Translator(service_urls=["translate.googleapis.com"])
        # self.img_labels = [
        #    r translator.translate(label).text for label in tqdm(annotation_df.basestyle.values)
        # ]
        self.img_labels = df.basestyle.values
        # self.img_labels, self.label_encoding = pd.factorize(self.img_labels)
        self.img_dir_path = df.img_dir_path
        self.filename = df.filename
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image_path = os.path.join(self.img_dir_path[idx], self.filename[idx])
        # image = io.read_image(image_path)
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        label = self.img_labels[idx]

        return image, label

In [6]:
# Combine the labels from both dataframes
all_labels = np.r_[train_df.basestyle.values, val_df.basestyle.values]

# Use pd.factorize to get the encodings and indices
img_labels, label_encoding = pd.factorize(all_labels)

# Replace the labels in train_df with their indices
train_df["basestyle"] = img_labels[: train_df.shape[0]]

# Replace the labels in test_df with their indices
val_df["basestyle"] = img_labels[train_df.shape[0] :]

In [7]:
def decode_label(label, label_encoding):
    return label_encoding[label]

In [8]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

train_ds = HairStyleDataset(train_df, transform)
val_ds = HairStyleDataset(val_df, transform)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=True)

# Train

In [9]:
num_classes = label_encoding.shape[0]

# Load the ResNet18 model and replace its classifier head
model = resnet18(weights=ResNet18_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(512, num_classes)

# model.fc.requires_grad = True

In [10]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

model.to(device);

In [11]:
# 2. Save model inputs and hyperparameters
config = wandb.config
wandb.watch(model)

[]

In [12]:
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()

# params_to_update = [model]
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [13]:
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for batch_idx, (inputs, labels) in enumerate(tqdm(dataloader, leave=False)):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        if batch_idx % 10 == 0:
            wandb.log({"Training Running Loss": loss})
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.cpu().double() / len(dataloader.dataset)
    wandb.log({"Training Loss": epoch_loss, "Training Accuracy": epoch_acc})

In [14]:
def test(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    for batch_idx, (inputs, labels) in enumerate(tqdm(dataloader, leave=False)):
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.cpu().double() / len(dataloader.dataset)
    wandb.log({"Validation Loss": epoch_loss, "Validation Accuracy": epoch_acc})

In [15]:
%%wandb

num_epochs = 25
for epoch in trange(num_epochs):
    train(model, train_dl, criterion, optimizer)
    test(model, val_dl, criterion)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1207 [00:00<?, ?it/s]

/Users/astadnik/.virtualenvs/masters/lib/python3.10/site-packages/wandb/wandb_torch.py:231: UserWarning: The operator 'aten::histc' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  tensor = flat.histc(bins=self._num_bins, min=tmin, max=tmax)


  0%|          | 0/302 [00:00<?, ?it/s]

  0%|          | 0/1207 [00:00<?, ?it/s]

...

  0%|          | 0/302 [00:00<?, ?it/s]

  0%|          | 0/1207 [00:00<?, ?it/s]

  0%|          | 0/302 [00:00<?, ?it/s]

  0%|          | 0/1207 [00:00<?, ?it/s]

  0%|          | 0/302 [00:00<?, ?it/s]

In [16]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
import netron
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

[Adding "Freeze layers" for transfer learning](https://github.com/ultralytics/ultralytics/issues/562)

In [ ]:
model.info(verbose=True)

In [ ]:
model.export(format="onnx")

In [ ]:
netron.start("yolov8n.onnx")

[FINETUNING TORCHVISION MODELS](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)